In [1]:
import torch
from torch import optim, nn, utils
import numpy as np
import layers
import lenet5_cifar10 as models
from torchvision import datasets, transforms

seed = 2022
torch.manual_seed(2022)

In [2]:
transform = transforms.ToTensor()
root = "./CIFAR10_DATASET"
train_dataset = datasets.CIFAR10(root, transform=transform, train=True, download=True)
train_dataset, valid_dataset = utils.data.random_split(train_dataset, [40000, 10000])
test_dataset = datasets.CIFAR10(root, transform=transform, train=False, download=True)

print("Train : ", len(train_dataset))
print("Validation : ", len(valid_dataset))
print("Test : ", len(test_dataset))

train_batchsize = 64
test_batchsize = 256

train_dataloader = utils.data.DataLoader(train_dataset, batch_size=train_batchsize, shuffle=True)
valid_dataloader = utils.data.DataLoader(valid_dataset, batch_size=test_batchsize, shuffle=False)
test_dataloader = utils.data.DataLoader(test_dataset, batch_size=test_batchsize, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
Train :  40000
Validation :  10000
Test :  10000


In [10]:
def cifar10_experiment(bnn_model, train_dataloader, valid_dataloader, use_aleatoric=True, **kwargs):
  if bnn_model == "gaussian":
    model = models.gaussian_lenet5(kwargs["var_type"], dict(), kwargs["is_lrt"], use_aleatoric)
    if torch.cuda.is_available():
      model = model.cuda()
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.1 ** (1/100))
    # Every forward gives different output.
    bnn_type = "random"
  elif bnn_model == "dropout":
    model = models.dropout_lenet5(kwargs["dropout_rate"], kwargs["dropout_type"], dict(), use_aleatoric)
    if torch.cuda.is_available():
      model = model.cuda()
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"], weight_decay=(1 - kwargs["dropout_rate"]) / 2 * kwargs["num_sample"])
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.1 ** (1/100))
    # Every forward gives different output.
    bnn_type = "random"
  elif bnn_model == "ensemble":
    model = models.ensemble_lenet5(kwargs["num_ensemble"], use_aleatoric)
    if torch.cuda.is_available():
      model = model.cuda()
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.1 ** (1/100))
    # Forward gives [batch_size * num_ensemble, output_shape]
    bnn_type = "ensemble"
  elif bnn_model == "swag":
    model = models.lenet5(use_aleatoric)
    if torch.cuda.is_available():
      model = model.cuda()
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.1 ** (1/100))
    # SWAG is trained as simple NN.
    bnn_type = "swag"
  elif bnn_model == "batchensemble":
    model = models.batchensemble_lenet5(kwargs["num_models"], use_aleatoric)
    if torch.cuda.is_available():
      model = model.cuda()
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.1 ** (1/100))
    # Forward gives [batch_size * num_models, output_shape]
    bnn_type = "ensemble"
    kwargs["num_ensemble"] = kwargs["num_models"]
  else:
    raise ValueError("No bnn model choosen.")

  criterion = nn.CrossEntropyLoss()
  train_loss_res = []
  valid_loss_res = []
  train_acc_res = []
  valid_acc_res = []
  for epoch in range(kwargs["epoch"]):
    train_loss = 0.0
    train_acc_count = 0
    model.train()
    for data in train_dataloader:
      images, labels = data
      if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
      if bnn_type == "random":
        for ind_sample in range(kwargs["num_sample"]):
          outputs = model(images)
          if use_aleatoric:
            output_mean, output_std = torch.chunk(outputs, 2, dim=1)
            eps = torch.normal(0, 1, output_mean.shape, device='cuda' if torch.cuda.is_available() else 'cpu')
            outputs = output_mean + eps * output_std
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          train_loss += loss.item() / kwargs["num_sample"]
          output_pred = torch.argmax(outputs.detach(), dim=1)
          train_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["num_sample"]
          
      elif bnn_type == "ensemble":
        outputs = model(images)
        if use_aleatoric:
          output_mean, output_std = torch.chunk(outputs, 2, dim=1)
          eps = torch.normal(0, 1, output_mean.shape, device='cuda' if torch.cuda.is_available() else 'cpu')
          outputs = output_mean + eps * output_std
        labels = labels.repeat(kwargs["num_ensemble"]) # [y1, y2, ..., y1, y2, ..., ] with num_ensemble times
        loss = criterion(outputs, labels) * kwargs["num_ensemble"]
        loss.backward()
        optimizer.step()

        train_loss += loss.item() / kwargs["num_ensemble"]
        output_pred = torch.argmax(outputs.detach(), dim=1)
        train_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["num_ensemble"]
      else:
        outputs = model(images)
        if use_aleatoric:
          output_mean, output_std = torch.chunk(outputs, 2, dim=1)
          eps = torch.normal(0, 1, output_mean.shape, device='cuda' if torch.cuda.is_available() else 'cpu')
          outputs = output_mean + eps * output_std
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        output_pred = torch.argmax(outputs.detach(), dim=1)
        train_acc_count += torch.count_nonzero(output_pred == labels).item()
    train_loss_res.append(train_loss)
    train_acc_res.append(train_acc_count / len(train_dataset))

    model.eval()
    valid_loss = 0.0
    valid_acc_count = 0
    with torch.no_grad():
      for data in valid_dataloader:
        images, labels = data
        if torch.cuda.is_available():
          images = images.cuda() 
          labels = labels.cuda()
        if bnn_type == "random":
          for ind_sample in range(kwargs["valid_num_sample"]):
            outputs = model(images)
            if use_aleatoric:
              output_mean, output_std = torch.chunk(outputs, 2, dim=1)
              eps = torch.normal(0, 1, output_mean.shape, device='cuda' if torch.cuda.is_available() else 'cpu')
              outputs = output_mean + eps * output_std
            loss = criterion(outputs, labels)
            
            valid_loss += loss.item() / kwargs["valid_num_sample"]
            output_pred = torch.argmax(outputs.detach(), dim=1)
            valid_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["valid_num_sample"]
        elif bnn_type == "ensemble":
          outputs = model(images)
          if use_aleatoric:
            output_mean, output_std = torch.chunk(outputs, 2, dim=1)
            eps = torch.normal(0, 1, output_mean.shape, device='cuda' if torch.cuda.is_available() else 'cpu')
            outputs = output_mean + eps * output_std
          labels = labels.repeat(kwargs["num_ensemble"]) # [y1, y2, ..., y1, y2, ..., ] with num_ensemble times
          loss = criterion(outputs, labels) * kwargs["num_ensemble"]
          valid_loss += loss.item() / kwargs["num_ensemble"]
          output_pred = torch.argmax(outputs.detach(), dim=1)
          valid_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["num_ensemble"]
        else:
          outputs = model(images)
          if use_aleatoric:
            output_mean, output_std = torch.chunk(outputs, 2, dim=1)
            eps = torch.normal(0, 1, output_mean.shape, device='cuda' if torch.cuda.is_available() else 'cpu')
            outputs = output_mean + eps * output_std
          loss = criterion(outputs, labels)
          valid_loss += loss.item()
          output_pred = torch.argmax(outputs.detach(), dim=1)
          valid_acc_count += torch.count_nonzero(output_pred == labels).item()
    valid_loss_res.append(valid_loss)
    valid_acc_res.append(valid_acc_count / len(valid_dataset))
  
  return model, train_loss_res, train_acc_res, valid_loss_res, valid_acc_res

In [5]:
res = dict()
g_model_sq, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("gaussian", train_dataloader, valid_dataloader, var_type='sq', is_lrt=True, num_sample=5, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["g_sq"] = (tr_los, tr_ac, te_los, te_ac)
model, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("gaussian", train_dataloader, valid_dataloader, var_type='exp', is_lrt=True, num_sample=5, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["g_exp"] = (tr_los, tr_ac, te_los, te_ac)

In [6]:
d_model_w, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("dropout", train_dataloader, valid_dataloader, dropout_rate=0.2, dropout_type='w', num_sample=5, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["d_w"] = (tr_los, tr_ac, te_los, te_ac)
d_model_f, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("dropout", train_dataloader, valid_dataloader, dropout_rate=0.2, dropout_type='f', num_sample=5, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["d_f"] = (tr_los, tr_ac, te_los, te_ac)
d_model_c, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("dropout", train_dataloader, valid_dataloader, dropout_rate=0.2, dropout_type='c', num_sample=5, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["d_c"] = (tr_los, tr_ac, te_los, te_ac)

In [11]:
e_model, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("ensemble", train_dataloader, valid_dataloader, num_ensemble = 5, epoch=5, lr=0.00001, momentum=0.1)
res["e"] = (tr_los, tr_ac, te_los, te_ac)

In [12]:
model, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("swag", train_dataloader, valid_dataloader, epoch=5, lr=0.00001, momentum=0.1)
res["s"] = (tr_los, tr_ac, te_los, te_ac)

In [13]:
b_model, tr_los, tr_ac, te_los, te_ac = cifar10_experiment("batchensemble", train_dataloader, valid_dataloader, num_models=5, epoch=5, lr=0.00001, momentum=0.1)

In [15]:
for k, v in res.items():
  print(k, v[0])

g_sq [1448.8034969091395, 1313.0767204999931, 1263.7788180112807, 1206.261340260507, 1161.5435376167259]
g_exp [11757.193000411977, 8924.014388847354, 5629.349616241436, 3346.8385259151437, 2244.986705780036]
d_w [1406.1137231826756, 1296.0705082178122, 1238.433707475662, 1213.6736639738074, 1183.8800067663217]
d_f [1379.1210598230336, 1265.2037019968052, 1203.4538650274294, 1180.36409025192, 1172.8353870153412]
d_c [1404.3464307069803, 1316.2657466888436, 1282.413014459608, 1265.7925008296966, 1255.4905275821707]
e [1440.3204744338989, 1432.8018617630003, 1398.9832727432258, 1332.905601119995, 1305.190331840516]
s [1440.7577877044678, 1436.5418717861176, 1424.1227571964264, 1347.0442192554474, 1311.9374333620071]
